In [1]:
import pandas as pd

df = pd.read_csv("aggregated.csv")

In [2]:
df.head(3)

,MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,FL_NUM,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,CRS_DEP_TIME,ARR_DEL15,CRS_ELAPSED_TIME,DISTANCE,Unnamed: 13
0,2.0,6.0,2017-02-25,B6,28.0,MCO,"Orlando, FL",EWR,"Newark, NJ",1000.0,0.0,156.0,937.0,NaN
1,2.0,7.0,2017-02-26,B6,28.0,MCO,"Orlando, FL",EWR,"Newark, NJ",739.0,0.0,153.0,937.0,NaN
2,2.0,1.0,2017-02-27,B6,28.0,MCO,"Orlando, FL",EWR,"Newark, NJ",1028.0,0.0,158.0,937.0,NaN


In [3]:
# print(len(df))
df = df.iloc[:,:-1] #Ignoring last column (Unname: 13)
print(len(df))
# df = df.dropna()
# print(len(df))

# a = df['ARR_DEL15'] != 0
# b = df[df['ARR_DEL15'] != 1]

5129354


In [ ]:
# import numpy as np

# totaldelays = np.sum(a)
# # totalnodelays = np.sum(b)
# # print(totaldelays, totalnodelays)

# b = b.iloc[:,:totaldelays]

In [ ]:
# print(len(b))

In [4]:
from sklearn import preprocessing

FL_DATE = df.iloc[:,2]
UNIQUE_CARRIER = df.iloc[:,3]
ORIGIN = df.iloc[:,5]
ORIGIN_CITY_NAME = df.iloc[:,6]
DEST = df.iloc[:,7]
DEST_CITY_NAME = df.iloc[:,8]
# UNIQUE_CARRIER.head(5), ORIGIN.head(5), ORIGIN_CITY_NAME.head(5), DEST.head(5), DEST_CITY_NAME.head(5)

In [5]:
le = preprocessing.LabelEncoder()

le.fit(FL_DATE)
encoded_FL_DATE = le.transform(FL_DATE)

le.fit(UNIQUE_CARRIER)
encoded_UNIQUE_CARRIER = le.transform(UNIQUE_CARRIER)

le.fit(ORIGIN)
encoded_ORIGIN = le.transform(ORIGIN)

le.fit(ORIGIN_CITY_NAME)
encoded_ORIGIN_CITY_NAME = le.transform(ORIGIN_CITY_NAME)

le.fit(DEST)
encoded_DEST = le.transform(DEST)

le.fit(DEST_CITY_NAME)
encoded_DEST_CITY_NAME = le.transform(DEST_CITY_NAME)

df['FL_DATE'] = encoded_FL_DATE
df['UNIQUE_CARRIER'] = encoded_UNIQUE_CARRIER
df['ORIGIN'] = encoded_ORIGIN
df['ORIGIN_CITY_NAME'] = encoded_ORIGIN_CITY_NAME
df['DEST'] = encoded_DEST
df['DEST_CITY_NAME'] = encoded_DEST_CITY_NAME

In [6]:
df.head(3)

,MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,FL_NUM,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,CRS_DEP_TIME,ARR_DEL15,CRS_ELAPSED_TIME,DISTANCE
0,2.0,6.0,330,2,28.0,188,219,98,208,1000.0,0.0,156.0,937.0
1,2.0,7.0,331,2,28.0,188,219,98,208,739.0,0.0,153.0,937.0
2,2.0,1.0,332,2,28.0,188,219,98,208,1028.0,0.0,158.0,937.0


In [7]:
# Splitting into train data and train labels
X = df.iloc[:,df.columns != 'ARR_DEL15']
# X = X.iloc[:,:-1] #Ignoring last column (Unname: 13)
Y = df.iloc[:,df.columns == 'ARR_DEL15']
print(len(X), len(Y))
# X = X.dropna()
# Y = Y.dropna()
# print(len(X), len(Y))

5129354 5129354


In [19]:
split = int(len(Y) * 0.8)
train_X = X[:split]
train_Y = Y[:split]
test_X = X[split:]
test_Y = Y[split:]

from sklearn.preprocessing import Imputer

imputer = Imputer(missing_values = 'NaN', strategy = 'most_frequent', axis = 0)
train_x_imputed = imputer.fit_transform(train_X)
test_x_imputed = imputer.fit_transform(test_X)

train_y_imputed = imputer.fit_transform(train_Y)
test_y_imputed = imputer.fit_transform(test_Y)

print('training lengths :',len(train_x_imputed), len(train_y_imputed))
print('testing lengths :',len(test_x_imputed), len(test_y_imputed))

training lengths : 4103483 4103483
testing lengths : 1025871 1025871


In [20]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(train_x_imputed, train_y_imputed)
hyp = clf.predict(test_x_imputed)

from sklearn.metrics import accuracy_score

accuracy_score(hyp, test_y_imputed)

0.7957335766387782

In [125]:
from sklearn.linear_model import LogisticRegression

C = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]

for c in C:
    clf = LogisticRegression(penalty = 'l1', C=c)
    clf.fit(train_x_imputed, train_y_imputed)
    hyp = clf.predict(test_x_imputed)
    accuracy = accuracy_score(test_y_imputed,hyp)
    print('C = ', c, 'Accuracy : ', accuracy)

/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


C =  0.0001 Accuracy :  0.8291442101394815
C =  0.001 Accuracy :  0.8291442101394815
C =  0.01 Accuracy :  0.8291442101394815
C =  0.1 Accuracy :  0.8291442101394815
C =  1 Accuracy :  0.8291442101394815
C =  10 Accuracy :  0.8291442101394815
C =  100 Accuracy :  0.8291442101394815
C =  1000 Accuracy :  0.8291442101394815
C =  10000 Accuracy :  0.8291442101394815


In [126]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier()
clf.fit(train_x_imputed, train_y_imputed)
hyp = clf.predict(test_x_imputed)
print("Accuracy Score :",accuracy_score(test_y_imputed, hyp))

/usr/local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy Score : 0.8291442101394815


In [11]:
from sklearn.model_selection import KFold
import warnings; warnings.simplefilter('ignore')
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# train_data = train_data.values.tolist()
# train_labels = train_labels.values.tolist()

accuracylist = []

x_for_kfold = X
kfold = KFold(n_splits=10)
for train_index, test_index in kfold.split(x_for_kfold):
#     print('Train: {} | Test: {}'.format(train_index,test_index))
    X_train, X_test = X.values[train_index], X.values[test_index]
    y_train, y_test = Y.values[train_index], Y.values[test_index]
#     print(len(X_train), len(X_test))
    imputer = Imputer(missing_values = 'NaN', strategy = 'most_frequent', axis = 0)
    train_x_imputed = imputer.fit_transform(X_train)
    test_x_imputed = imputer.fit_transform(y_train)
    train_y_imputed = imputer.fit_transform(X_test)
    test_y_imputed = imputer.fit_transform(y_test)
#     clf = LogisticRegression()
    clf = RandomForestClassifier()
    clf.fit(train_x_imputed, test_x_imputed)
    hyp = clf.predict(train_y_imputed)
    accuracylist.append(accuracy_score(test_y_imputed, hyp))
    print("Accuracy Score :",accuracy_score(test_y_imputed, hyp))
print("10 fold cross validation Accuracy Score :", np.mean(accuracylist))

Accuracy Score : 0.8164273905516478
Accuracy Score : 0.839952742642357
Accuracy Score : 0.7793876818940375
Accuracy Score : 0.8264617808069623
Accuracy Score : 0.8040122042753955
Accuracy Score : 0.7527698441322974
Accuracy Score : 0.7757142717888231
Accuracy Score : 0.8075077738894791
Accuracy Score : 0.8382017214656828
Accuracy Score : 0.763874565003363
10 fold cross validation Accuracy Score : [0.8164273905516478, 0.839952742642357, 0.7793876818940375, 0.8264617808069623, 0.8040122042753955, 0.7527698441322974, 0.7757142717888231, 0.8075077738894791, 0.8382017214656828, 0.763874565003363]


In [16]:
import numpy as np
print("10 fold cross validation Accuracy Score :", np.mean(accuracylist))
print(sum(hyp==1))

10 fold cross validation Accuracy Score : 0.8004309976450046
39880


In [155]:
from sklearn.model_selection import KFold
import warnings; warnings.simplefilter('ignore')

# train_data = train_data.values.tolist()
# train_labels = train_labels.values.tolist()

accuracylist = []

x_for_kfold = X
kfold = KFold(n_splits=10)
for train_index, test_index in kfold.split(x_for_kfold):
#     print('Train: {} | Test: {}'.format(train_index,test_index))
    X_train, X_test = X.values[train_index], X.values[test_index]
    y_train, y_test = Y.values[train_index], Y.values[test_index]
#     print(len(X_train), len(X_test))
    imputer = Imputer(missing_values = 'NaN', strategy = 'most_frequent', axis = 0)
    train_x_imputed = imputer.fit_transform(X_train)
    test_x_imputed = imputer.fit_transform(y_train)
    train_y_imputed = imputer.fit_transform(X_test)
    test_y_imputed = imputer.fit_transform(y_test)
    clf = LogisticRegression()
#     clf = RandomForestClassifier()
    clf.fit(train_x_imputed, test_x_imputed)
    hyp = clf.predict(train_y_imputed)
    accuracylist.append(accuracy_score(test_y_imputed, hyp))
    print("Accuracy Score :",accuracy_score(test_y_imputed, hyp))
print("10 fold cross validation Accuracy Score :", np.mean(accuracylist))

Accuracy Score : 0.8478855061840074
Accuracy Score : 0.8494549027558994
Accuracy Score : 0.7897359514637303
Accuracy Score : 0.8493242821716549


KeyboardInterrupt: 

In [22]:
from sklearn.metrics import accuracy_score, confusion_matrix

print('Confusion Matrix:\n', confusion_matrix(test_y_imputed, hyp))

Confusion Matrix:
 [[794311  56284]
 [153267  22009]]


In [ ]:
# DecisionTree = 0.7766
# RF = 0.7964
# LR = 0.8291
# MLP = 0.8291